In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
import pandas as pd
import numpy as np

df_final = pd.read_csv('/content/drive/MyDrive/BDA_project/df_final.csv')
# test_data = pd.read_csv('/content/drive/MyDrive/BDA_project/test.csv')

In [4]:
df_final.drop(columns = 'Unnamed: 0', inplace =True)

In [5]:
df_final

,id,date,route_id,vh_id,route_nm,now_station,now_arrive_time,distance,next_station,next_arrive_time,week,s/m,m/s,time
0,0,2019-10-15,405136001,7997025,360-1,제주대학교입구,06시,266.000000,제대마을,24.0,Tuesday,0.090226,11.083333,morning
1,1,2019-10-15,405136001,7997025,360-1,제대마을,06시,333.265436,제대아파트,36.0,Tuesday,0.108108,9.250000,morning
2,2,2019-10-15,405136001,7997025,360-1,제대아파트,06시,420.903308,제주대학교,40.0,Tuesday,0.096386,10.375000,morning
3,3,2019-10-15,405136001,7997025,360-1,남국원(아라방면),06시,578.000000,제주여자중고등학교(아라방면),42.0,Tuesday,0.072664,13.761905,morning
4,4,2019-10-15,405136001,7997025,360-1,도호동,07시,322.581998,은남동,64.0,Tuesday,0.171123,5.843750,morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203756,210452,2019-10-28,405328102,7983486,281-2,비석거리,21시,526.655462,삼아아파트,96.0,Monday,0.181818,5.500000,evening
203757,210453,2019-10-28,405328102,7983486,281-2,동문로터리,21시,280.000000,매일올레시장 7번입구,50.0,Monday,0.178571,5.600000,evening
203758,210454,2019-10-28,405328102,7983486,281-2,서귀포시 구 버스터미널,21시,114.000000,아랑조을거리 입구,16.0,Monday,0.140351,7.125000,evening
203759,210455,2019-10-28,405328102,7983486,281-2,아랑조을거리 입구,21시,219.256538,평생학습관,38.0,Monday,0.170404,5.868421,evening


In [6]:
from sklearn.preprocessing import LabelEncoder # 라벨 인코더

station_encoder = LabelEncoder() # 인코더 생성

_station = list(df_final['now_station'].values) + list(df_final['next_station'].values) # train_data 의 모든 정류장 이름
station_set = set(_station)
#print(len(station_set))

station_encoder.fit(list(station_set)) # 인코딩

# 모든 학습, 시험 데이터의 정류장 정보 치환
df_final['now_station'] = station_encoder.transform(df_final['now_station']) #train data
df_final['next_station'] = station_encoder.transform(df_final['next_station'])

In [7]:
df_final = pd.get_dummies(df_final,columns=['time']) # 원 핫 인코딩을 수행
df_final.head()

,id,date,route_id,vh_id,route_nm,now_station,now_arrive_time,distance,next_station,next_arrive_time,week,s/m,m/s,time_afternoon,time_evening,time_morning
0,0,2019-10-15,405136001,7997025,360-1,239,06시,266.000000,227,24.0,Tuesday,0.090226,11.083333,0,0,1
1,1,2019-10-15,405136001,7997025,360-1,227,06시,333.265436,228,36.0,Tuesday,0.108108,9.250000,0,0,1
2,2,2019-10-15,405136001,7997025,360-1,228,06시,420.903308,235,40.0,Tuesday,0.096386,10.375000,0,0,1
3,3,2019-10-15,405136001,7997025,360-1,41,06시,578.000000,249,42.0,Tuesday,0.072664,13.761905,0,0,1
4,4,2019-10-15,405136001,7997025,360-1,63,07시,322.581998,221,64.0,Tuesday,0.171123,5.843750,0,0,1


In [8]:
## train data

df_final['date'] = pd.to_datetime(df_final['date']) # date 값을 datetime으로
df_final['week'] = df_final['date'].dt.weekday  # Monday 0, Sunday 6
df_final['week'] = df_final['week'].apply(lambda x: 0 if x <= 5 else 1)
# 0 ~ 5 는 월요일 ~ 금요일이므로 평일이면 0, 주말이면 1을 설정하였다

train_data = pd.get_dummies(df_final, columns=['week']) # 평일/주말에 대해 One-hot Encoding

train_data = train_data.drop('date', axis=1) # 필요없는 date 칼럼을 drop

## train data

train_data = train_data.drop(['id', 'now_arrive_time', 'route_nm'], axis=1)

train_data.head()

,route_id,vh_id,now_station,distance,next_station,next_arrive_time,s/m,m/s,time_afternoon,time_evening,time_morning,week_0,week_1
0,405136001,7997025,239,266.000000,227,24.0,0.090226,11.083333,0,0,1,1,0
1,405136001,7997025,227,333.265436,228,36.0,0.108108,9.250000,0,0,1,1,0
2,405136001,7997025,228,420.903308,235,40.0,0.096386,10.375000,0,0,1,1,0
3,405136001,7997025,41,578.000000,249,42.0,0.072664,13.761905,0,0,1,1,0
4,405136001,7997025,63,322.581998,221,64.0,0.171123,5.843750,0,0,1,1,0


In [9]:
test_data = pd.read_csv('/content/drive/MyDrive/BDA_project/test.csv')

In [10]:
test_data = test_data.drop(columns=['now_latitude','now_longitude','next_latitude','next_longitude'])

test_data['time'] = 'time'

test_data.loc[ (test_data['now_arrive_time'] >='05시') & (test_data['now_arrive_time'] <'12시') ,['time'] ]= 'morning' # 05~11시
test_data.loc[ (test_data['now_arrive_time'] >='12시') & (test_data['now_arrive_time'] <'18시') ,['time'] ]= 'afternoon' # 12~17시
test_data.loc[ (test_data['now_arrive_time'] >='18시') | (test_data['now_arrive_time'] =='00시') ,['time'] ]= 'evening' # 18~0시

In [11]:
test_data['now_station'] = station_encoder.transform(test_data['now_station']) #test data
test_data['next_station'] = station_encoder.transform(test_data['next_station'])


In [12]:
## test data
# 시험데이터도 마찬가지로 처리해준다.

test_data['date'] = pd.to_datetime(test_data['date'])
test_data['week'] = test_data['date'].dt.weekday  # Monday 0, Sunday 6
test_data['week'] = test_data['week'].apply(lambda x: 0 if x <= 5 else 1)
test_data = pd.get_dummies(test_data,columns=['time'])
test_data = pd.get_dummies(test_data, columns=['week'])

test_data = test_data.drop('now_arrive_time', axis=1)
test_data = test_data.drop('date', axis=1)

## test data

test_data = test_data.drop(['route_nm'], axis=1)
test_data.head()

,id,route_id,vh_id,now_station,distance,next_station,time_afternoon,time_evening,time_morning,week_0,week_1
0,210457,405136001,7997025,227,333.0,228,0,0,1,1,0
1,210458,405136001,7997025,228,415.0,235,0,0,1,1,0
2,210459,405136001,7997025,323,417.0,59,0,0,1,1,0
3,210460,405136001,7997025,59,334.0,183,0,0,1,1,0
4,210461,405136001,7997025,183,550.0,285,0,0,1,1,0


In [13]:
train_data.columns

Index(['route_id', 'vh_id', 'now_station', 'distance', 'next_station',
       'next_arrive_time', 's/m', 'm/s', 'time_afternoon', 'time_evening',
       'time_morning', 'week_0', 'week_1'],
      dtype='object')

In [14]:
input_var = list(train_data.columns)
input_var.remove('next_arrive_time')
input_var.remove('m/s')
input_var.remove('s/m')

Xtrain = train_data[input_var] # 학습 데이터 선택
Ytrain = train_data['next_arrive_time'] # target 값인 Y 데이터 선택

Xtest = test_data[input_var] # 시험 데이터도 선택

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_valid, Y_train, Y_valid = train_test_split(Xtrain, Ytrain, test_size=0.2, random_state=156)

In [16]:
from sklearn.model_selection import GridSearchCV

def get_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid=params, scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_train, Y_train)
    rmse = np.sqrt(-1* grid_model.best_score_)
    print('최적 평균 RMSE 값:', np.round(rmse, 4))
    print('최적 파라미터:', grid_model.best_params_)

    return grid_model.best_estimator_

In [17]:
import xgboost as xgb

# xgb_params 값을 바꾸어주며 learning_rate, max_depth 등 파라미터 성능 비교 시도했었음.
xgb_params = {'n_estimators':[100, 200, 980, 1000, 2500, 2750, 3300]}
#'max_depth':[3,5,7,9], 'min_child_weight':[1,3], 'colsample_bytree':[0.5,0.75],
xgb_reg = xgb.XGBRegressor()

best_xgb = get_best_params(xgb_reg, xgb_params)

최적 평균 RMSE 값: 25.7
최적 파라미터: {'n_estimators': 200}


In [18]:
best_xgb.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 200,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [19]:
from lightgbm import LGBMRegressor

# lgbm_params 값을 바꾸어주며 learning_rate, max_depth 등 파라미터 성능 비교 시도했었음.
lgbm_params = {'n_estimators':[100, 200, 450, 980, 1000, 2500, 2750, 3300]}

lgbm_reg = LGBMRegressor()

best_lgbm = get_best_params(lgbm_reg, lgbm_params)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003731 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 859
[LightGBM] [Info] Number of data points in the train set: 130406, number of used features: 10
[LightGBM] [Info] Start training from score 79.871578
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002114 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 858
[LightGBM] [Info] Number of data points in the train set: 130406, number of used features: 10
[LightGBM] [Info] Start training from score 80.063666
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002670 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not eno

In [20]:
best_lgbm

LGBMRegressor(n_estimators=1000)